References:
    - https://github.com/tkipf/keras-gcn/

In [1]:
from __future__ import print_function

In [2]:
# graph convolution
from keras import activations, initializers, constraints
from keras import regularizers
from keras.engine import Layer
import keras.backend as K

Using TensorFlow backend.


In [3]:
class GraphConvolution(Layer):
    """ Graph convolution layer as in https://arxiv.org/abs/1609.02907"""
    
    def __init__(self, 
                 units, 
                 support=1,
                activation=None,
                use_bias=True,
                kernel_initializer='glorot_uniform',
                bias_initializer='zeros',
                kernel_regularizer=None,
                bias_regularizer=None,
                activity_regularizer=None,
                kernel_constraint=None,
                bias_constraint=None,
                **kwargs):
        assert support >= 1
        
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'), )
        
        super(GraphConvolution, self).__init__(**kwargs)
        self.units = units
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.supports_masking = True
        
        self.support = support
    
    def compute_output_shape(self, input_shapes):
        features_shape = input_shapes[0]
        output_shape = (features_shape[0], self.units)
        return output_shape
    
    def build(self, input_shapes):
        features_shape = input_shapes[0]
#         print(features_shape)
        assert len(features_shape) == 2
        input_dim = features_shape[1]
        
        self.kernel = self.add_weight(shape=(input_dim * self.support, self.units),
                                     initializer=self.kernel_initializer,
                                     name='kernel',
                                     regularizer=self.kernel_regularizer,
                                     constraint=self.kernel_constraint)
        
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units, ),
                                       initializer= self.bias_initializer,
                                       name='bias',
                                       regularizer=self.bias_regularizer,
                                       constraint=self.bias_constraint)
        else:
            self.bias = None
        self.built = True
        
    def call(self, inputs, mask=None):
        features = inputs[0]
        basis = inputs[1:]
        
        supports = list()
        for i in range(self.support):
            supports.append(K.dot(basis[i], features))
        supports = K.concatenate(supports, axis=1)
        output = K.dot(supports, self.kernel)
        
        if self.bias:
            output += self.bias
        return self.activation(output)
    
    def get_config(self):
        config = {'units': self.units,
               'support': self.support,
               'activiation': activations.serialize(self.activation),
               'use_bias': self.use_bias,
               'kernel_initializer': initializers.serialize(self.kernel_initializer),
               'bias_initializer': initializers.serialize(self.bias_initializer),
               'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
                'bias_regularizer': regularizers.serialize(self.bias_regularizer),
                'activity_regularizer': regularizers.serialize(self.activity_regularizer),
                'kernel_constraint': constraints.serialize(self.kernel_constraint),
                'bias_constraint':constraints.serialize(self.bias_constraint)
               }
        
        
        base_config = super(GraphConvolution, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [4]:
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

In [5]:
import time

In [6]:
DATASET = 'cora'
FILTER = 'localpool' #chebyshev
MAX_DEGREE = 2
SYM_NORM = True # symmetric
NB_EPOCH = 200
PATIENCE = 10 # early stopping

In [7]:
import scipy.sparse as sp
import numpy as np

def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

def load_data(path='./datasets/CORA_origin/', dataset='cora'):
    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset), dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])
    
    #graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset), dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])), shape=(labels.shape[0], labels.shape[0]), dtype=np.float32)
    
    adj = adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    print('Dataset has {} nodes, {} edges, {} features.'.format(adj.shape[0], edges.shape[0], features.shape[1]))
    
    return features.todense(), adj, labels

In [8]:
X, A, y = load_data(dataset=DATASET)

Dataset has 2708 nodes, 5429 edges, 1433 features.


In [9]:
def sample_mask(idx, l):
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)

def get_splits(y):
    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)
    y_train = np.zeros(y.shape, dtype=np.int32)
    y_val = np.zeros(y.shape, dtype=np.int32)
    y_test = np.zeros(y.shape, dtype=np.int32)
    y_train[idx_train] = y[idx_train]
    y_val[idx_val] = y[idx_val]
    y_test[idx_test] = y[idx_test]
    train_mask = sample_mask(idx_train, y.shape[0])
    return y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask

In [10]:
# ratio of labels
140/2708 *100

5.1698670605613

In [11]:
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

In [12]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [13]:
# normalize
X /= X.sum(1).reshape(-1, 1)

In [14]:
from scipy.sparse.linalg.eigen.arpack import eigsh, ArpackNoConvergence

def normalize_adj(adj, symmetric=True):
    if symmetric:
        d = sp.diags(np.power(np.array(adj.sum(1)), -0.5).flatten(), 0)
        a_norm = adj.dot(d).transpose().dot(d).tocsr()
    else:
        d = sp.diags(np.power(np.array(adj.sum(1)), -1).flatten(), 0)
        a_norm = adj.dot(d).transpose().dot(d).tocsr()
    return a_norm

def preprocess_adj(adj, symmetric=True):
    adj = adj + sp.eye(adj.shape[0])
    return normalize_adj(adj, symmetric)

def normalized_laplacian(adj, symmetric=True):
    adj_n = normalize_adj(adj, symmetric)
    laplacian = sp.eye(adj.shape[0]) - adj_n
    return laplacian

def rescale_laplacian(laplacian):
    try:
        print('Calculating largest eigenvalue of normalized graph laplacian')
        largest_eigenval = eigsh(laplacian, 1, which='LM', return_eigenvectors=False)[0]
    except ArpackNoConvergence:
        print('Eigevnvalue calculation did not converge! Using largest_eigenvaaal=2 instead.')
        largest_eigenval = 2
        
    scaled_laplacian = (2. / largest_eigenval) * laplacian - sp.eye(laplacian.shape[0])
    return scaled_laplacian

def chebyshev_polynomial(X, k):
    """ calculate chebyshev polynomials up to order k"""
    T_k = list()
    T_k.append(sp.eye(X.shape[0]).tocsr())
    T_k.append(X)
    
    def chebyshev_recurrence(T_k_minus_one, T_k_minus_two, X):
        X_ = sp.csr_matrix(X, copy=True)
        return 2 * X_.dot(T_k_minus_one) - T_k_minus_two
    
    for i in range(2, k+1):
        T_k.append(chebyshev_recurrence(T_k[-1], T_k[-2], X))
    
    return T_k

In [15]:
if FILTER == 'localpool':
    """ renormalization trick in Kipf & Welling, arXiv 2016 """
    A_ = preprocess_adj(A, SYM_NORM)
    support = 1
    graph = [X, A_]
    G = [Input(shape=(None, None), batch_shape=(None, None))]

elif FILTER == 'chebyshev':
    """ chebyshev Defferard et al. 2016"""
    L = normalized_laplacian(A, SYM_NORM)
    L_scaled = rescale_laplacian(L)
    T_k = chebyshev_polynomial(L_scaled, MAX_DEGREE)
    support = MAX_DEGREE + 1
    graph = [X] + T_k
    G = [Input(shape=(None, None), batch_shape=(None, None)) for i in range(support)]

else:
    raise Exception('Invalid filter type')

W0715 11:45:43.509906 139835335409792 deprecation_wrapper.py:119] From /home/madigun/.pyenv/versions/3.7.3/envs/annon/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0715 11:45:43.518188 139835335409792 deprecation_wrapper.py:119] From /home/madigun/.pyenv/versions/3.7.3/envs/annon/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [16]:
X_in = Input(shape=(X.shape[1], ))

In [17]:
# model architecture
# pass arguments for graph convolutional layers as a list of tensors
H = Dropout(0.5)(X_in)
H = GraphConvolution(20, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

W0715 11:45:43.528494 139835335409792 deprecation_wrapper.py:119] From /home/madigun/.pyenv/versions/3.7.3/envs/annon/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0715 11:45:43.533531 139835335409792 deprecation.py:506] From /home/madigun/.pyenv/versions/3.7.3/envs/annon/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0715 11:45:43.582360 139835335409792 deprecation_wrapper.py:119] From /home/madigun/.pyenv/versions/3.7.3/envs/annon/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [18]:
# compile
model = Model(inputs=[X_in] + G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

W0715 11:45:43.628032 139835335409792 deprecation_wrapper.py:119] From /home/madigun/.pyenv/versions/3.7.3/envs/annon/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0715 11:45:43.633022 139835335409792 deprecation_wrapper.py:119] From /home/madigun/.pyenv/versions/3.7.3/envs/annon/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [19]:
wait = 0
preds = None
best_val_loss = 99999

In [20]:
def categorical_crossentropy(preds, labels):
    return np.mean(-np.log(np.extract(labels, preds)))

def accuracy(preds, labels):
    return np.mean(np.equal(np.argmax(labels, 1), np.argmax(preds, 1)))

def evaluate_preds (preds, labels, indices):
    split_loss = list()
    split_acc = list()
    
    for y_split, idx_split, in zip(labels, indices):
        split_loss.append(categorical_crossentropy(preds[idx_split], y_split[idx_split]))
        split_acc.append(accuracy(preds[idx_split], y_split[idx_split]))
        
    return split_loss, split_acc

In [21]:
for epoch in range(1, NB_EPOCH + 1):
    t = time.time()
    
    model.fit(graph, y_train, sample_weight=train_mask,
             batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)
    
    # predict full dataset
    preds = model.predict(graph, batch_size=A.shape[0])
    
    # train/ validation
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val], [idx_train, idx_val])
    
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))
    
    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1
        
print('time for training {}'.format(str(time.time() - t)))

W0715 11:45:43.718542 139835335409792 deprecation.py:323] From /home/madigun/.pyenv/versions/3.7.3/envs/annon/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 0001 train_loss= 1.9278 train_acc= 0.4143 val_loss= 1.9365 val_acc= 0.3133 time= 0.6058
Epoch: 0002 train_loss= 1.9101 train_acc= 0.4071 val_loss= 1.9279 val_acc= 0.3600 time= 0.1779
Epoch: 0003 train_loss= 1.8911 train_acc= 0.4286 val_loss= 1.9195 val_acc= 0.3767 time= 0.1710
Epoch: 0004 train_loss= 1.8708 train_acc= 0.4500 val_loss= 1.9112 val_acc= 0.3933 time= 0.1646
Epoch: 0005 train_loss= 1.8491 train_acc= 0.4643 val_loss= 1.9023 val_acc= 0.4267 time= 0.1752
Epoch: 0006 train_loss= 1.8270 train_acc= 0.4500 val_loss= 1.8935 val_acc= 0.4200 time= 0.1672
Epoch: 0007 train_loss= 1.8049 train_acc= 0.4357 val_loss= 1.8851 val_acc= 0.3967 time= 0.1716
Epoch: 0008 train_loss= 1.7843 train_acc= 0.4143 val_loss= 1.8786 val_acc= 0.3900 time= 0.1803
Epoch: 0009 train_loss= 1.7648 train_acc= 0.4143 val_loss= 1.8732 val_acc= 0.3900 time= 0.1671
Epoch: 0010 train_loss= 1.7456 train_acc= 0.4357 val_loss= 1.8687 val_acc= 0.3933 time= 0.1683
Epoch: 0011 train_loss= 1.7276 train_acc= 0.4571 v

In [22]:
# Testing
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print("Test set results:",
      "loss= {:.4f}".format(test_loss[0]),
      "accuracy= {:.4f}".format(test_acc[0]))

Test set results: loss= 1.4783 accuracy= 0.4720
